# Foundation Models para Dados Tabulares
## Comparacao: XGBoost vs TabPFN

### Trade-off: Performance vs Interpretabilidade

| Modelo | Performance | Interpretabilidade |
|--------|-------------|-------------------|
| XGBoost + SHAP | Boa | **Excelente** |
| TabPFN | **Excelente** | Limitada |

> **Em credit scoring, explicabilidade e requisito regulatorio (LGPD, BACEN).**

### Requisito: HuggingFace Token
TabPFN v6+ requer autenticacao. Configure `HF_TOKEN` como variavel de ambiente.

In [1]:
import sys
sys.path.insert(0, '../src')

import os
# Configure seu token: os.environ['HF_TOKEN'] = 'seu_token'

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

from credit_scoring.data.loader import load_german_credit
from credit_scoring.models.train import load_model

print("Setup completo!")

Setup completo!


In [2]:
X, y = load_german_credit(save_raw=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Treino: {len(X_train)} | Teste: {len(X_test)}")

Treino: 800 | Teste: 200


## XGBoost (Baseline)

In [3]:
xgb_model = load_model('../models/best_model_optuna.joblib')
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
xgb_pred = xgb_model.predict(X_test)

xgb_metrics = {
    'Model': 'XGBoost (Optuna)',
    'ROC-AUC': roc_auc_score(y_test, xgb_proba),
    'F1-Score': f1_score(y_test, xgb_pred),
    'Precision': precision_score(y_test, xgb_pred),
    'Recall': recall_score(y_test, xgb_pred),
    'Interpretable': 'Sim (SHAP)'
}
print(f"XGBoost ROC-AUC: {xgb_metrics['ROC-AUC']:.4f}")

XGBoost ROC-AUC: 0.8058


## TabPFN (Foundation Model)

In [4]:
from tabpfn import TabPFNClassifier

X_train_p = xgb_model.named_steps['preprocessor'].transform(X_train)
X_test_p = xgb_model.named_steps['preprocessor'].transform(X_test)

print("Inicializando TabPFN...")
tabpfn = TabPFNClassifier()

print("Treinando...")
tabpfn.fit(X_train_p, y_train)

print("Fazendo predicoes...")
tabpfn_proba = tabpfn.predict_proba(X_test_p)[:, 1]
tabpfn_pred = tabpfn.predict(X_test_p)

tabpfn_metrics = {
    'Model': 'TabPFN',
    'ROC-AUC': roc_auc_score(y_test, tabpfn_proba),
    'F1-Score': f1_score(y_test, tabpfn_pred),
    'Precision': precision_score(y_test, tabpfn_pred),
    'Recall': recall_score(y_test, tabpfn_pred),
    'Interpretable': 'Limitada'
}
print(f"TabPFN ROC-AUC: {tabpfn_metrics['ROC-AUC']:.4f}")

Inicializando TabPFN...
Treinando...
Fazendo predicoes...
TabPFN ROC-AUC: 0.8142


## Comparacao Final

In [5]:
results = [xgb_metrics, tabpfn_metrics]
comparison_df = pd.DataFrame(results).sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 60)
print("COMPARACAO: XGBoost vs TabPFN")
print("=" * 60)
print(comparison_df.to_string(index=False))


COMPARACAO: XGBoost vs TabPFN
            Model   ROC-AUC  F1-Score  Precision  Recall Interpretable
           TabPFN  0.814167  0.576923   0.681818     0.5      Limitada
XGBoost (Optuna)  0.805833  0.657534   0.558140     0.8    Sim (SHAP)


In [6]:
fig, ax = plt.subplots(figsize=(10, 4))

models = comparison_df['Model'].tolist()
auc_scores = comparison_df['ROC-AUC'].tolist()
colors = ['#9b59b6' if 'TabPFN' in m else '#2ecc71' for m in models]

bars = ax.barh(models, auc_scores, color=colors, edgecolor='black')
for bar, score in zip(bars, auc_scores):
    ax.text(score + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{score:.4f}', va='center', fontweight='bold')

ax.set_xlabel('ROC-AUC Score', fontsize=12)
ax.set_title('XGBoost vs TabPFN (Foundation Model)', fontsize=14, fontweight='bold')
ax.set_xlim(0.70, 0.90)

plt.tight_layout()
plt.savefig('../reports/figures/foundation_models_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

comparison_df.to_csv('../reports/foundation_models_comparison.csv', index=False)
print("Salvo em: reports/foundation_models_comparison.csv")

Salvo em: reports/foundation_models_comparison.csv


## Conclusao

| Modelo | ROC-AUC | Recomendacao |
|--------|---------|-------------|
| **TabPFN** | **0.8142** | Prototipo/Benchmark |
| XGBoost | 0.8058 | **Producao (SHAP)** |

TabPFN superou XGBoost em ROC-AUC, mas XGBoost oferece interpretabilidade essencial para compliance.